# 9장 소프트웨어 인터페이스
## 목표

- 웹 클라이언트(requests)
- 데이터 직렬화(pickle)
- AMQP(pick, celery)

# 참고: jupyter notebook 에서 가상환경 분리하기!

저는 hitchhiker라는 가상환경을 생성한 후 사용하고 있습니다.

1. 가상환경 생성(virtualenvwrapper를 사용하면 간편합니다!)
2. terminal에서 'jupyter --paths' 입력 후, data의 Jupyter 경로 확인 (ex./Users/bagmunsu/Library/Jupyter)
3. 1번에서 생성한 가상환경을 활성화하여 'which python'을 통해 가상환경 위치 파악
4. 2번에서 확인한 Jupyter에 kernel 추가, 터미널에 'mkdir /Users/bagmunsu/Library/Jupyter/kernels/(가상환경명)'
5. 4번에서 생성한 경로에 'kernel.json' 추가
    - kernel.json 예시
    ~~~json
    {
    "argv": [ "/Users/bagmunsu/.virtualenvs/study/bin/python", "-m", "ipykernel",
              "-f", "{connection_file}"],
    "display_name": "hitchhikers",
    "language": "python"
    }
    ~~~
6. Jupyter를 다시 실행하여 우측 상단 "New" 버튼에 추가한 커널을 확인한다

jupyter notebook 에서 아래와 같이 입력해서 설치된 패키지를 확인해 볼 수 있습니다.

```
!pip list
```

## 웹 클라이언트 & json 파싱
- [Request](http://docs.python-requests.org/en/master/) 모듈은 HTTP 요청에 대한 모든 작업을 수행하여, 웹에서 데이터를 얻는데 도움을 준다.
- Requests 라이브러리의 Response 객체에는 내장 json 파서가 있어서, json 형식의 문자열이나 파일을 파이썬 딕셔너리(혹은 리스트)로 파싱한다.


In [34]:
import requests

# 코인원 API URL, http://doc.coinone.co.kr/#api-Public
api_url_home = 'https://api.coinone.co.kr/'

# 코인원 메인 페이지 시세 확인하기
home_result = requests.get(api_url_home)
print("Coinone home api GET 결과")
print("type: {}".format(type(home_result)))
print("text 포맷: \n{}\n".format(home_result.text))


Coinone home api GET 결과
type: <class 'requests.models.Response'>
text 포맷: 
<html>
	<head>
		<title>Coinone API</title>
	</head>
	<body>
		<script>
            window.location.href="http://doc.coinone.co.kr/";
		</script>
	</body>
</html>
	



In [35]:
# GET 에 parameter를 넘겨야 하는 경우
# 코인별 전날 가격 정보 보기
def print_yesterday_price_info(coin_name):
    """
    
    코인원의 전날 코인가격 정보를 프린트합니다.
    API 문서: http://doc.coinone.co.kr/#api-Public-Ticker_UTC
    
    Args:
        coin_name: 코인이름

    """
    if not coin_name:
        coin_name = 'btc'
    result = requests.get(
        'https://api.coinone.co.kr/ticker_utc/',
        params={'currency': coin_name}
    ) # ticker: 증권 시세 표시기
    print('--------------------------------------------------')
    print("Coinone ticker api GET 결과({})\n".format(coin_name))
    print("type: {}\n".format(type(result)))
    result_text = result.text
    result_json = result.json()
    print("text 포맷({}): \n{}\n".format(type(result_text), result_text))
    print("json 형태({}): \n{}\n".format(type(result_json), result_json))

    print("전일 최고가: {:,}, 전일 최저가: {:,}, 전일 종가: {:,}".format(
        int(result_json['yesterday_high']),
        int(result_json['yesterday_low']),
        int(result_json['yesterday_last'])
    ))
    print('--------------------------------------------------')

print_yesterday_price_info('btc')
print_yesterday_price_info('eth')

--------------------------------------------------
Coinone ticker api GET 결과(btc)

type: <class 'requests.models.Response'>

text 포맷(<class 'str'>): 
{"result":"success","volume":"1236.2766","last":"9188000","yesterday_last":"9512000","timestamp":"1526455369","yesterday_low":"9485000","high":"9527000","currency":"btc","low":"9070000","errorCode":"0","yesterday_first":"9587000","yesterday_volume":"1169.5985","yesterday_high":"9690000","first":"9512000"}

json 형태(<class 'dict'>): 
{'result': 'success', 'volume': '1236.2766', 'last': '9188000', 'yesterday_last': '9512000', 'timestamp': '1526455369', 'yesterday_low': '9485000', 'high': '9527000', 'currency': 'btc', 'low': '9070000', 'errorCode': '0', 'yesterday_first': '9587000', 'yesterday_volume': '1169.5985', 'yesterday_high': '9690000', 'first': '9512000'}

전일 최고가: 9,690,000, 전일 최저가: 9,485,000, 전일 종가: 9,512,000
--------------------------------------------------
--------------------------------------------------
Coinone ticker api GET 결

## 데이터 직렬화

- 데이터 직렬화(serializion)란 구조화된 데이터를 공유하거나 저장할 수 있는 형식으로 변환하는 개념
- ![직렬화 이미지](https://docs.google.com/drawings/d/1OP2TLGYP0DLolFhQ3yxeeJ6S7MYi3iLOKg-e_kN9nKU/pub?w=919&h=124)
- 파이썬 기본 데이터 직렬화 모듈은 [pickle](https://docs.python.org/3.6/library/pickle.html)이다. 

In [38]:
import pickle

grades = {'손흥민': 9, '기성용': 8, '이승우': 5, '조현우': 7}

serial_grades = pickle.dumps(grades)

print(serial_grades)

received_grades = pickle.loads(serial_grades)

print(received_grades)

b'\x80\x03}q\x00(X\t\x00\x00\x00\xec\x86\x90\xed\x9d\xa5\xeb\xaf\xbcq\x01K\tX\t\x00\x00\x00\xea\xb8\xb0\xec\x84\xb1\xec\x9a\xa9q\x02K\x08X\t\x00\x00\x00\xec\x9d\xb4\xec\x8a\xb9\xec\x9a\xb0q\x03K\x05X\t\x00\x00\x00\xec\xa1\xb0\xed\x98\x84\xec\x9a\xb0q\x04K\x07u.'
{'손흥민': 9, '기성용': 8, '이승우': 5, '조현우': 7}
